In [1]:
import sys, platform, subprocess
print("Python:", platform.python_version())      # p.ej., 3.11.9
print("Version info:", sys.version_info)         # tuple detallada
print("Executable:", sys.executable)             # ruta del intérprete
subprocess.run([sys.executable, "-m", "pip", "--version"])  # pip correspondiente

Python: 3.10.12
Version info: sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0)
Executable: /usr/bin/python3
pip 25.2 from /home/marcelo/.local/lib/python3.10/site-packages/pip (python 3.10)


CompletedProcess(args=['/usr/bin/python3', '-m', 'pip', '--version'], returncode=0)

## Preliminares

In [2]:
import dspy
import importlib.metadata

info = importlib.metadata.metadata("dspy")

print("Nombre:", info["Name"])
print("Versión:", info["Version"])
print("Email autor:", info["Author-email"])
print("Licencia:", info["License"])
print("Resumen:", info["Summary"])

Nombre: dspy
Versión: 2.6.27
Email autor: Omar Khattab <okhattab@stanford.edu>
Licencia: MIT License
Resumen: DSPy


In [3]:
class SolveSignature(dspy.Signature):
    """Resuelve el problema paso a paso y entrega una respuesta final."""
    question: str = dspy.InputField()
    rationale: str = dspy.OutputField(desc="razonamiento paso a paso")  # CoT
    answer: str = dspy.OutputField(desc="respuesta final, breve")

## M1

In [4]:
# Cargamos un llm
lm1 = dspy.LM('openai/gpt-4o', api_key='?')

dspy.configure(lm=lm1)

# Módulo que induce razonamiento paso a paso
solver = dspy.ChainOfThought(SolveSignature)

# Demo (few-shot) con razonamiento explícito que irá en el prompt
demo = dspy.Example(
    question="Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?",
    rationale="Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls, 5 + 6 = 11.",
    answer="11"
).with_inputs("question")  # 'question' es el/los campo(s) de entrada

# Adjuntar la demo al predictor (se incluye en el prompt al inferir)
solver.demos = [demo]   # puedes apilar varias demos en esta lista


In [5]:
q = "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"
pred_1 = solver(question=q)

print("Razonamiento (CoT):", pred_1.rationale)
print("Respuesta:", pred_1.answer)

Razonamiento (CoT): 1. Start with the initial number of apples: 23.
2. Subtract the number of apples used for lunch: 23 - 20 = 3.
3. Add the number of apples bought: 3 + 6 = 9.
4. The final number of apples the cafeteria has is 9.
Respuesta: 9


## M2

In [6]:
# Cargamos un llm
lm2 = dspy.LM('openai/gpt-4o-mini', api_key='?)

dspy.configure(lm=lm2)
# Módulo que induce razonamiento paso a paso
solver = dspy.ChainOfThought(SolveSignature)
solver.demos = [demo]
pred_2 = solver(question=q)

print("Razonamiento (CoT):", pred_2.rationale)
print("Respuesta:", pred_2.answer)

Razonamiento (CoT): 1. Initial apples: 23
2. Apples used: 20
3. Apples bought: 6

Calculating step by step:
- After using 20 apples: 23 - 20 = 3 apples remaining.
- After buying 6 more apples: 3 + 6 = 9 apples total.
Respuesta: The cafeteria has 9 apples.


## M3

In [8]:
# Cargamos un llm
lm3 = dspy.LM('openai/gpt-4.1-mini', api_key='?')

dspy.configure(lm=lm3)
# Módulo que induce razonamiento paso a paso
solver = dspy.ChainOfThought(SolveSignature)
solver.demos = [demo]
pred_3 = solver(question=q)

print("Razonamiento (CoT):", pred_3.rationale)
print("Respuesta:", pred_3.answer)

Razonamiento (CoT): Step 1: Start with 23 apples.
Step 2: Subtract the 20 apples used for lunch: 23 - 20 = 3 apples remaining.
Step 3: Add the 6 apples bought: 3 + 6 = 9 apples in total.
Respuesta: 9 apples


## Juez

In [9]:
# Cada candidato: {'model': str, 'answer': str, 'rationale_summary': str}
candidates = [
    {"model": "M1", "answer": pred_1.answer, "rationale_summary": pred_1.rationale},
    {"model": "M2", "answer": pred_2.answer, "rationale_summary": pred_2.rationale},
    {"model": "M3", "answer": pred_3.answer, "rationale_summary": pred_3.rationale},
]


In [29]:
candidates

[{'model': 'M1',
  'answer': '9',
  'rationale_summary': '1. Start with the initial number of apples: 23.\n2. Subtract the number of apples used for lunch: 23 - 20 = 3.\n3. Add the number of apples bought: 3 + 6 = 9.\n4. The final number of apples the cafeteria has is 9.'},
 {'model': 'M2',
  'answer': 'The cafeteria has 9 apples.',
  'rationale_summary': '1. Initial apples: 23\n2. Apples used: 20\n3. Apples bought: 6\n\nCalculating step by step:\n- After using 20 apples: 23 - 20 = 3 apples remaining.\n- After buying 6 more apples: 3 + 6 = 9 apples total.'},
 {'model': 'M3',
  'answer': '9 apples',
  'rationale_summary': 'Step 1: Start with 23 apples.\nStep 2: Subtract the 20 apples used for lunch: 23 - 20 = 3 apples remaining.\nStep 3: Add the 6 apples bought: 3 + 6 = 9 apples in total.'}]

In [10]:
import json
from typing import List

# ---------- Signatures ----------
class JudgeSignature(dspy.Signature):
    """Evalúa y selecciona la mejor respuesta."""
    question: str = dspy.InputField(desc="la pregunta original")
    candidates_json: str = dspy.InputField(desc="lista JSON con objetos {model, answer, rationale_summary}")
    rubric: str = dspy.InputField(desc=("criterios y ponderaciones; p.ej., "
                                        "Corrección(0.5), Consistencia(0.2), "
                                        "Completitud(0.2), Seguridad(0.1)"))
    # Salidas (breves y estructuradas)
    selected_index: int = dspy.OutputField(desc="índice del mejor candidato (0..N-1)")
    scores_json: str = dspy.OutputField(desc="JSON con puntajes por candidato y por criterio")
    brief_justification: str = dspy.OutputField(desc="justificación concisa (≤3 frases)")

class SynthesizeSignature(dspy.Signature):
    """Produce una respuesta final unificada."""
    question: str = dspy.InputField()
    topk_json: str = dspy.InputField(desc="JSON con los mejores candidatos (1–2) y sus puntos fuertes/débiles")
    final_answer: str = dspy.OutputField(desc="respuesta unificada, clara y coherente")
    brief_reasoning: str = dspy.OutputField(desc="justificación breve (≤3 frases)")


In [11]:
class JudgeModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.judge = dspy.Predict(JudgeSignature)  # puedes cambiar a dspy.ChainOfThought si deseas razonamiento interno

    def forward(self, question: str, candidates: List[dict], rubric: str):
        j = self.judge(
            question=question,
            candidates_json=json.dumps(candidates, ensure_ascii=False),
            rubric=rubric
        )
        # Parseo seguro
        try:
            scores = json.loads(j.scores_json)
        except Exception:
            scores = {}
        return dict(
            selected_index=int(j.selected_index),
            scores=scores,
            brief_justification=j.brief_justification.strip()
        )

class SynthesizeModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.synth = dspy.Predict(SynthesizeSignature)

    def forward(self, question: str, topk: List[dict]):
        s = self.synth(
            question=question,
            topk_json=json.dumps(topk, ensure_ascii=False)
        )
        return dict(
            final_answer=s.final_answer.strip(),
            brief_reasoning=s.brief_reasoning.strip()
        )


In [15]:
rubric = '''Evalúa cada candidato con esta rúbrica y devuelve JSON estructurado:
- Corrección factual (0.5)
- Consistencia lógica (0.2)
- Completitud (0.2)
- Seguridad/No alucinar (0.1)

Instrucciones:
1) Asigna a cada criterio una nota de 1–5.
2) Calcula score_total ponderado por candidato.
3) Elige selected_index = argmax(score_total).
4) Devuelve scores_json como: 
   {"candidates":[{"model":"...", "scores":{"correccion":5,...}, "total":4.3}, ...]}
5) brief_justification: máx 3 frases, sin pasos detallados.
'''

In [17]:
judge = JudgeModule()
synth = SynthesizeModule()
question = q

# Ejecuta el juez
jout = judge(question, candidates, rubric)

# Elige top-k (p. ej., 2 mejores por total)
ranked = sorted(
    jout["scores"].get("candidates", []),
    key=lambda x: x.get("total", 0),
    reverse=True
)
topk = []
for c in ranked[:2]:
    # Recupera el texto original del candidato a partir del 'model' o índice
    for i, cand in enumerate(candidates):
        if cand["model"] == c.get("model"):
            topk.append({
                "model": cand["model"],
                "answer": cand["answer"],
                "rationale_summary": cand["rationale_summary"],
                "strengths": "Alta corrección y consistencia" if c.get("total",0) >= 4 else "Equilibrado",
                "weaknesses": "Completitud mejorable" if c.get("total",0) < 4 else ""
            })
            break

# Síntesis final
sout = synth(question, topk)

final_answer = sout["final_answer"]
judge_note   = jout["brief_justification"]  # máx 3 frases
synth_note   = sout["brief_reasoning"]      # máx 3 frases

In [18]:
print("="*60)
print("🧑‍⚖️  Nota del Juez:")
print(judge_note)
print("-"*60)
print("🧩  Nota de Síntesis:")
print(synth_note)
print("-"*60)
print("✅  Respuesta Final:")
print(final_answer)
print("="*60)

🧑‍⚖️  Nota del Juez:
Los tres candidatos ofrecen respuestas correctas, completas y consistentes sin errores ni alucinaciones. La diferencia entre ellos es mínima y solo en la forma de presentación, por lo que cualquiera es válido. Se selecciona el primero por ser claro y directo.
------------------------------------------------------------
🧩  Nota de Síntesis:
Starting with 23 apples, they used 20, leaving 3. Then they bought 6 more, resulting in 3 + 6 = 9 apples.
------------------------------------------------------------
✅  Respuesta Final:
The cafeteria has 9 apples.


## ReConcile

In [30]:
import re

# -----------------------------
# Utilidades "deterministas"
# -----------------------------
def parse_int_answer(s: str):
    """Extrae el primer entero de la cadena (e.g., '9', '9 apples', 'The answer is 9')."""
    m = re.search(r"(-?\d+)", str(s))
    return int(m.group(1)) if m else None

def validate_apples_word_problem(question: str):
    """
    Verificador específico para el patrón 'had X, used Y, bought Z'.
    Retorna (respuesta_correcta, pasos_texto)
    """
    # Para este problema concreto basta con parsear X=23, Y=20, Z=6
    nums = list(map(int, re.findall(r"\d+", question)))
    # Si quieres algo más general, podrías mapear verbos a +/-.
    if len(nums) >= 3:
        X, Y, Z = nums[:3]
        after_use = X - Y
        final = after_use + Z
        steps = [f"{X} - {Y} = {after_use}", f"{after_use} + {Z} = {final}"]
        return final, steps
    return None, []

def majority_vote_int(ints):
    """Devuelve el entero más común; si hay empate, devuelve el primero que aparece."""
    from collections import Counter
    c = Counter(ints)
    top = c.most_common()
    return top[0][0]

# -----------------------------
# 1) Normalización por candidato (DSPy)
# -----------------------------
class NormalizeCandidateSig(dspy.Signature):
    """Extrae una respuesta numérica y una ecuación corta a partir de la salida del modelo."""
    question: str = dspy.InputField()
    answer: str = dspy.InputField(desc="texto de respuesta del candidato")
    rationale_summary: str = dspy.InputField(desc="resumen breve del razonamiento del candidato")

    # Salidas (breves y públicas, no CoT interno)
    numeric_answer: str = dspy.OutputField(desc="entero en texto, p.ej. '9'")
    short_equation: str = dspy.OutputField(desc="ecuación corta tipo '23 - 20 + 6 = 9'")

class NormalizeCandidate(dspy.Module):
    def __init__(self): 
        super().__init__()
        self.m = dspy.Predict(NormalizeCandidateSig)
    def forward(self, question, candidate):
        out = self.m(
            question=question,
            answer=candidate["answer"],
            rationale_summary=candidate["rationale_summary"]
        )
        # fallback robusto: si no vino número, lo parseamos nosotros
        num = parse_int_answer(out.numeric_answer) if out.numeric_answer else parse_int_answer(candidate["answer"])
        return {
            "model": candidate["model"],
            "numeric_answer": num,
            "short_equation": out.short_equation.strip() if out.short_equation else ""
        }

# -----------------------------
# 2) ReConcile (síntesis conciliatoria)
# -----------------------------
class ReconcileSig(dspy.Signature):
    """Redacta una respuesta final unificada con razonamiento conciliatorio breve."""
    question: str = dspy.InputField()
    candidates_json: str = dspy.InputField(desc="candidatos normalizados con numeric_answer y short_equation")
    majority_answer: str = dspy.InputField(desc="entero final por mayoría (texto)")
    validator_steps: str = dspy.InputField(desc="pasos verificados, p.ej. '23 - 20 = 3; 3 + 6 = 9'")

    final_answer: str = dspy.OutputField(desc="respuesta final (solo el número si corresponde, p.ej. '9')")
    conciliatory_rationale: str = dspy.OutputField(desc="3–5 frases: acuerda, explica el proceso y concluye")

class Reconcile(dspy.Module):
    def __init__(self):
        super().__init__()
        self.m = dspy.Predict(ReconcileSig)
    def forward(self, question, normalized_candidates, majority_answer, validator_steps):
        cj = json.dumps(normalized_candidates, ensure_ascii=False)
        vs = "; ".join(validator_steps)
        out = self.m(
            question=question,
            candidates_json=cj,
            majority_answer=str(majority_answer),
            validator_steps=vs
        )
        return {
            "final_answer": out.final_answer.strip(),
            "conciliatory_rationale": out.conciliatory_rationale.strip()
        }

# -----------------------------
# 3) Orquestación end-to-end
# -----------------------------
def reconcile_word_problem(question: str, candidates: list):
    # (a) Normalizar candidatos
    normalizer = NormalizeCandidate()
    normalized = [normalizer(question, c) for c in candidates]

    # (b) Voto por mayoría
    nums = [n["numeric_answer"] for n in normalized if n["numeric_answer"] is not None]
    majority = majority_vote_int(nums) if nums else None

    # (c) Verificador independiente (determinista)
    gold, steps = validate_apples_word_problem(question)

    # Si por alguna razón majority es None, usa el verificado.
    final_num = majority if majority is not None else gold

    # (d) Síntesis conciliatoria
    recon = Reconcile()
    out = recon(question, normalized, final_num, steps)

    # Si el LLM no devuelve el número “pelado”, forzamos a este valor seguro:
    if parse_int_answer(out["final_answer"]) is None and final_num is not None:
        out["final_answer"] = str(final_num)

    return {
        "normalized": normalized,
        "majority_answer": final_num,
        "validator_answer": gold,
        "validator_steps": steps,
        "final_answer": out["final_answer"],
        "conciliatory_rationale": out["conciliatory_rationale"]
    }


In [31]:
result = reconcile_word_problem(q, candidates)

In [32]:
def pretty_print_reconcile(result, title="ReConcile Report"):
    print("="*72)
    print(f"📘 {title}")
    print("="*72)

    # Tabla de candidatos normalizados
    print("\n📦 Normalización por modelo:")
    header = f"{'Modelo':<8} | {'Ans. num':<9} | {'Ecuación corta'}"
    print(header)
    print("-"*72)
    for it in result["normalized"]:
        model = it.get("model","?")
        num   = it.get("numeric_answer","?")
        eq    = it.get("short_equation","").strip()
        print(f"{model:<8} | {str(num):<9} | {eq}")
    print("-"*72)

    # Verificación determinista
    steps = result.get("validator_steps", [])
    if steps:
        print("\n🧮 Verificación independiente:")
        for i, st in enumerate(steps, 1):
            print(f"  {i:>2}. {st}")

    # Mayoría y final
    print("\n👥 Mayoría por respuesta:", result.get("majority_answer"))
    print("✅ Respuesta final:", result.get("final_answer"))

    # Razonamiento conciliatorio (síntesis pública)
    print("\n🧩 Razonamiento conciliatorio:")
    print(result.get("conciliatory_rationale","(sin texto)"))
    print("="*72)

pretty_print_reconcile(result, title="ReConcile Report")

📘 ReConcile Report

📦 Normalización por modelo:
Modelo   | Ans. num  | Ecuación corta
------------------------------------------------------------------------
M1       | 9         | 23 - 20 + 6 = 9
M2       | 9         | 23 - 20 + 6 = 9
M3       | 9         | 23 - 20 + 6 = 9
------------------------------------------------------------------------

🧮 Verificación independiente:
   1. 23 - 20 = 3
   2. 3 + 6 = 9

👥 Mayoría por respuesta: 9
✅ Respuesta final: 9

🧩 Razonamiento conciliatorio:
All models agree that the cafeteria started with 23 apples, used 20, leaving 3 apples, and then bought 6 more. Adding these 6 apples to the remaining 3 results in a total of 9 apples. The calculation steps are consistent and correctly applied, confirming that the final answer is 9.


In [33]:
class ReconciliatoryReasoningSig(dspy.Signature):
    """Explica brevemente cómo se llega a la respuesta final (estilo juez)."""
    question: str = dspy.InputField()
    normalized_json: str = dspy.InputField(desc="[{model,numeric_answer,short_equation}]")
    majority_answer: str = dspy.InputField(desc="entero por mayoría (texto)")
    validator_answer: str = dspy.InputField(desc="entero verificado (texto)")
    validator_steps: str = dspy.InputField(desc="pasos verificados en texto; separados por '; '")
    policy: str = dspy.InputField(desc="guía de estilo (3–5 frases, sin CoT interno)")

    conciliatory_rationale: str = dspy.OutputField(desc="justificación breve (3–5 frases)")
    decision_summary: str = dspy.OutputField(desc="1 frase con el motivo principal")

class ReconciliatoryJudge(dspy.Module):
    def __init__(self):
        super().__init__()
        self.m = dspy.Predict(ReconciliatoryReasoningSig)

    def forward(self, question, normalized, majority_answer, validator_answer, validator_steps, policy):
        nj = json.dumps(normalized, ensure_ascii=False)
        vs = "; ".join(validator_steps or [])
        out = self.m(
            question=question,
            normalized_json=nj,
            majority_answer=str(majority_answer),
            validator_answer=str(validator_answer),
            validator_steps=vs,
            policy=policy
        )
        return dict(
            conciliatory_rationale=out.conciliatory_rationale.strip(),
            decision_summary=out.decision_summary.strip()
        )

RECONCILE_JUDGE_POLICY = """
Escribe 3–5 frases, estilo juez explicativo:
- Menciona el consenso entre modelos y cómo se verificó aritméticamente.
- Si la mayoría y el verificador coinciden, dilo claramente.
- Incluye la ecuación breve que valida el resultado.
- No muestres pasos internos de chain-of-thought; usa lenguaje público y conciso.
"""

def compute_judge_reasoning(question, result):
    judge = ReconciliatoryJudge()
    return judge(
        question=question,
        normalized=result["normalized"],
        majority_answer=result["majority_answer"],
        validator_answer=result["validator_answer"],
        validator_steps=result["validator_steps"],
        policy=RECONCILE_JUDGE_POLICY
    )


In [34]:
rr = compute_judge_reasoning(q, result)
print("\n🧑‍⚖️ Razonamiento del juez (conciliatorio):")
print(rr["conciliatory_rationale"])
print("\n🧷 Resumen del juez:")
print(rr["decision_summary"])


🧑‍⚖️ Razonamiento del juez (conciliatorio):
Los tres modelos coinciden en que la respuesta correcta es 9, y esta conclusión fue verificada aritméticamente. El cálculo se realizó restando las 20 manzanas usadas de las 23 iniciales, resultando en 3, y luego sumando las 6 manzanas compradas, dando un total de 9. Tanto la mayoría como el verificador están de acuerdo en esta respuesta. La ecuación que valida este resultado es: 23 - 20 + 6 = 9.

🧷 Resumen del juez:
La respuesta final es 9, confirmada por consenso de modelos y verificación aritmética.
